# Miami Hands-On Lab: Snowflake Intelligence for Builders

---

**📅 Date:** Thursday, October 16, 2025 | 04:00 PM ET  
**📍 Location:** The LAB Miami, 2750 NW 3rd Ave, Suite 24, Miami, FL 33127

**🎯 Event Focus:** Interactive hands-on lab exploring Snowflake Cortex Intelligence capabilities

---

### 👥 Presented By

**Atalia Horenshtien** - Senior Consultant, AI Engineering | Hakkoda, an IBM Company [atalia.horenshtien@ibm.com](mailto:atalia.horenshtien@ibm.com)
**Jacob Scott** - Senior Consultant, AI Engineering | Hakkoda, an IBM Company [jacob.scott@ibm.com](mailto:jacob.scott@ibm.com)
**John DaCosta** - Senior Solution Engineer | Snowflake [john.dacosta@snowflake.com](mailto:john.dacosta@snowflake.com)

### 🤝 In Partnership With

**Snowflake** and **Hakkoda, an IBM Company**

---

### 📚 What You'll Learn

In this guided session, we'll walk you step-by-step through how to use Snowflake Intelligence for tangible business outcomes:
- Setting up your environment and connecting to live data
- Applying AI-powered features that surface insights faster
- Building Cortex Agents with RAG (Retrieval-Augmented Generation)
- Creating semantic models for natural language querying
- Integrating custom tools and external data sources

You'll leave with practical "how-to" skills, a repeatable framework, and the confidence to unlock intelligence from your own data with Snowflake and Hakkoda.

---

**🔗 Event Information:** [Miami Hands-On Lab Details](https://www.snowflake.com/event/miami-hands-on-lab-snowflake-intelligence-for-builders-20251016/)

---


### 📚 Snowflake Python Libraries Import

**What:** This cell imports essential Python libraries for working with Snowflake's advanced features including the Snowpark API and Core API.

**Why:** These libraries enable programmatic interaction with Snowflake, allowing you to:
- Create and manage Snowflake objects using the Core API (`snowflake.core`)
- Execute dataframe operations using Snowpark (`snowflake.snowpark`)
- Handle JSON data and work with DataFrames for data analysis

**How:** The imports provide access to:
- `json` - Parse JSON responses from Snowflake functions
- `pandas` - Data manipulation and analysis
- `snowflake.core.Root` - Entry point for Snowflake Core API to manage objects
- `snowflake.snowpark.Session` - Create Snowpark sessions for distributed computing

**📖 Documentation:**
- [Snowflake Python API Reference](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/index)
- [Snowpark Developer Guide](https://docs.snowflake.com/en/developer-guide/snowpark/python/index)
- [Snowflake Core API](https://docs.snowflake.com/en/developer-guide/snowflake-python-api/snowflake-python-overview)


In [ ]:
import json
import pandas as pd
import os
from snowflake.core import Root
from snowflake.snowpark import Session

# HOL Setup Notebook
Run the following commands to provision the necessary objects for the Snowflake Intelligence HOL

- add snowflake package


## Setup User Schema

### 🏗️ Schema Setup and Configuration

**What:** This cell sets up a personalized schema for each user within the HOL (Hands-On Lab) database using dynamic schema naming.

**Why:** 
- Creates isolated workspaces for multiple users in a shared training environment
- Uses the `IDENTIFIER()` function to dynamically create schemas based on the current user's name
- Prevents naming conflicts and allows each user to work independently

**How:** The code executes the following steps:
1. **USE ROLE HOL_ADMIN** - Switches to the administrator role with appropriate permissions
2. **SET CURRENT_USER = CURRENT_USER()** - Stores the logged-in username in a session variable
3. **USE DATABASE HOL** - Sets the working database context
4. **CREATE SCHEMA IF NOT EXISTS IDENTIFIER($CURRENT_USER)** - Creates a schema named after the user (e.g., if username is "JOHN_DOE", creates schema "JOHN_DOE")

The `IDENTIFIER()` function treats the variable value as an object identifier, enabling dynamic object creation.

**📖 Documentation:**
- [CREATE SCHEMA](https://docs.snowflake.com/en/sql-reference/sql/create-schema)
- [IDENTIFIER Function](https://docs.snowflake.com/en/sql-reference/identifier-literal)
- [Session Variables](https://docs.snowflake.com/en/sql-reference/session-variables)
- [USE ROLE](https://docs.snowflake.com/en/sql-reference/sql/use-role)


In [ ]:
USE ROLE HOL_ADMIN;

SET CURRENT_USER = CURRENT_USER();

USE DATABASE HOL;
CREATE SCHEMA IF NOT EXISTS IDENTIFIER($CURRENT_USER);


### 📂 Set Working Schema Context

**What:** This cell sets the active schema to your personal user schema for all subsequent operations.

**Why:** 
- Establishes the working context so all tables, views, and objects are created in your personal schema
- Eliminates the need to fully qualify object names in subsequent queries
- Ensures isolation between different users' work in the HOL environment

**How:** The `USE SCHEMA IDENTIFIER($CURRENT_USER)` command switches the session context to the schema created in the previous step, using the same dynamic identifier approach.

**📖 Documentation:**
- [USE SCHEMA](https://docs.snowflake.com/en/sql-reference/sql/use-schema)
- [Understanding Name Resolution](https://docs.snowflake.com/en/sql-reference/name-resolution)


In [ ]:
USE SCHEMA IDENTIFIER($CURRENT_USER);

## Setup Unstructured Chunks Table

### 🤖 Document Parsing and Text Chunking with AI

**What:** This cell performs intelligent document processing by:
1. Creating a table to store chunked text from documents
2. Using AI-powered OCR to extract text from PDF documents
3. Splitting extracted text into manageable chunks for semantic search

**Why:** 
- **Document Intelligence**: Converts unstructured PDF documents into searchable text using AI
- **Retrieval-Augmented Generation (RAG)**: Chunking enables efficient semantic search for RAG applications
- **Optimal Chunk Size**: 500-character chunks balance context preservation with search precision
- **Scalability**: Processes multiple documents from a stage automatically

**How:** The process involves four key steps:

1. **Create Table**: Defines schema for storing document chunks with metadata
   - `document_name` - Identifier for the source document
   - `chunk_id` - Index of the chunk within the document
   - `chunk_text` - The actual text content (max 500 chars)
   - `document_date` - Extracted date from document filename

2. **AI_PARSE_DOCUMENT**: Snowflake's AI-powered OCR function
   - Reads PDFs from the `@DATA.SEC_FILINGS_STAGE` 
   - `'mode': 'OCR'` - Uses optical character recognition for text extraction
   - `'page_split': false` - Keeps document as a single text block
   - Extracts text into the `:content` field

3. **SPLIT_TEXT_RECURSIVE_CHARACTER**: Cortex function for intelligent chunking
   - Splits long documents into 500-character segments
   - Preserves semantic meaning by respecting word boundaries
   - Returns an array of text chunks

4. **FLATTEN**: Converts array of chunks into rows for table storage

**📖 Documentation:**
- [AI_PARSE_DOCUMENT Function](https://docs.snowflake.com/en/user-guide/snowflake-cortex/document-ai/ai-parse-document)
- [SPLIT_TEXT_RECURSIVE_CHARACTER](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character)
- [FLATTEN Table Function](https://docs.snowflake.com/en/sql-reference/functions/flatten)
- [Document AI Overview](https://docs.snowflake.com/en/user-guide/snowflake-cortex/document-ai/document-ai-overview)


In [ ]:

-- CREATE OR REPLACE TABLE SEC_FILINGS_CHUNKS (
CREATE TABLE IF NOT EXISTS SEC_FILINGS_CHUNKS (
    document_name varchar,
    chunk_id varchar,
    chunk_text varchar,
    document_date date
);

-- Perform OCR and Chunk Text
INSERT INTO SEC_FILINGS_CHUNKS  
    with doc_text as (
        select
            split_part(relative_path,'.',0) as document_name,
            AI_PARSE_DOCUMENT (
                TO_FILE('@DATA.SEC_FILINGS_STAGE',relative_path),
                {'mode': 'OCR' , 'page_split': false}
            ):content::varchar as doc_text,
            to_date(split_part(document_name,'_',3),'MM-DD-YY') as doc_date
        from directory('@DATA.SEC_FILINGS_STAGE')
    )
    
    , chunked as (
        select 
            document_name,
            SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
              doc_text,
              'none',
              500
            ) as chunks,
            doc_date
        from doc_text
    )
    
    , flattened as (
        select
            document_name,
            index as chunk_id,
            value::varchar as chunk_text,
            doc_date
        from chunked,
            TABLE(FLATTEN(INPUT => CHUNKS))
    )
    
    SELECT * FROM flattened
;


### 🔍 Verify Chunked Document Data

**What:** This cell queries the SEC_FILINGS_CHUNKS table to examine the parsed and chunked document data.

**Why:** 
- Validates that documents were successfully processed
- Inspects the structure and quality of chunked text
- Confirms that metadata (document names, dates, chunk IDs) was correctly extracted

**How:** Simple SELECT statement retrieves all records from the table, showing the complete dataset of processed document chunks.

**📖 Documentation:**
- [SELECT Statement](https://docs.snowflake.com/en/sql-reference/sql/select)


In [ ]:
SELECT * FROM SEC_FILINGS_CHUNKS;

## Create Search Service for RAG

### 🔎 Create Cortex Search Service for Semantic Search

**What:** This cell creates a Cortex Search Service that enables semantic (meaning-based) search over the chunked documents and tests it with a sample query.

**Why:** 
- **Semantic Search**: Unlike keyword search, finds results based on meaning and context
- **RAG Foundation**: Essential component for Retrieval-Augmented Generation workflows
- **Vector Search**: Automatically creates embeddings for similarity search
- **Real-time Updates**: `TARGET_LAG` parameter keeps the index synchronized with table changes
- **Agent Integration**: Cortex Agents can use this service as a tool to retrieve relevant information

**How:** The process has two parts:

1. **CREATE CORTEX SEARCH SERVICE**:
   - `SEC_FILINGS_SEARCH` - Name of the search service
   - `ON CHUNK_TEXT` - The column containing text to be indexed and searched
   - `ATTRIBUTES DOCUMENT_DATE` - Additional filterable metadata columns
   - `WAREHOUSE = SNOWFLAKE_LEARNING_WH` - Compute resources for indexing
   - `TARGET_LAG = '7 DAYS'` - Maximum staleness for index updates
   - `AS (SELECT * FROM SEC_FILINGS_CHUNKS)` - Source data for the index

2. **Test Query**: The SEARCH_PREVIEW function demonstrates semantic search:
   - Searches for "NVIDIA revenue guidance" (natural language query)
   - Returns top 5 most relevant chunks with metadata
   - Results ranked by semantic similarity, not keyword matching
   - Returns JSON with matching document names, chunk IDs, dates, and text

**📖 Documentation:**
- [Cortex Search Service](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview)
- [CREATE CORTEX SEARCH SERVICE](https://docs.snowflake.com/en/sql-reference/sql/create-cortex-search-service)
- [SEARCH_PREVIEW Function](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/query-cortex-search-service)
- [Semantic Search Concepts](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview#how-cortex-search-works)


In [ ]:

CREATE CORTEX SEARCH SERVICE IF NOT EXISTS SEC_FILINGS_SEARCH
  ON CHUNK_TEXT
  ATTRIBUTES DOCUMENT_DATE
  WAREHOUSE = SNOWFLAKE_LEARNING_WH
  TARGET_LAG = '7 DAYS'
  AS (SELECT * FROM SEC_FILINGS_CHUNKS);

-- RUN A SAMPLE QUERY
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
    'SEC_FILINGS_SEARCH',
    '{
      "query": "NVIDIA revenue guidance",
      "columns": ["DOCUMENT_NAME", "CHUNK_ID", "DOCUMENT_DATE", "CHUNK_TEXT"],
      "limit": 5
    }'
  )
)['results'] AS results;

## Create a Custom Tool for the Agent
Just a dummy stock price tool, in reality this could easily be replaced with an API call using an external access integration.

### 🛠️ Create Custom Tool (Stored Procedure) for Agent

**What:** This cell creates a stored procedure that serves as a custom tool for the Cortex Agent, simulating a stock price lookup function.

**Why:** 
- **Agent Extensibility**: Demonstrates how to extend agent capabilities with custom functions
- **External Integration Pattern**: Shows the structure for tools that could call external APIs
- **Function Calling**: Cortex Agents can automatically detect when to call this tool based on user queries
- **Prototype/Demo**: This dummy implementation returns a fixed value, but in production could:
  - Call external APIs via External Access Integrations
  - Query real-time data feeds
  - Integrate with market data providers

**How:** The procedure is defined with:
- **Parameters**:
  - `TICKER STRING` - Stock ticker symbol (e.g., "NVDA", "AAPL")
  - `EXCHANGE STRING` - Stock exchange identifier (e.g., "NYSE", "NASDAQ")
- **Returns**: FLOAT - Stock price value
- **Language**: SQL (could also be Python, JavaScript, Java)
- **Logic**: Currently returns hardcoded value (185.88) for demonstration
- **Test Call**: Executes the procedure with sample parameters ('nvda', 'nyse')

**When added to a Cortex Agent**, the agent will:
1. Understand when users ask about stock prices
2. Automatically call this function with appropriate parameters
3. Incorporate the results into its response

**📖 Documentation:**
- [CREATE PROCEDURE](https://docs.snowflake.com/en/sql-reference/sql/create-procedure)
- [Stored Procedures Overview](https://docs.snowflake.com/en/developer-guide/stored-procedure/stored-procedures-overview)
- [Cortex Agent Custom Tools](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst/cortex-analyst-tools)
- [External Access Integration](https://docs.snowflake.com/en/developer-guide/external-network-access/external-network-access-overview)


In [ ]:
-- Dummy Tool
CREATE OR REPLACE PROCEDURE GET_STOCK_PRICE(TICKER STRING, EXCHANGE STRING)
RETURNS FLOAT
LANGUAGE SQL
AS
$$
DECLARE
    stock_price FLOAT;
BEGIN
    stock_price := 185.88;
    RETURN stock_price;
END;
$$;

call get_stock_price('nvda','nyse');

## Verify Existing Price History Data
For structured data we will be using a preloaded table made available for the user.

### 📊 Clone Price History Data Table

**What:** This cell creates a personal copy of the stock price history table using Snowflake's zero-copy cloning feature.

**Why:** 
- **Instant Copy**: Creates a full copy without duplicating storage or waiting for data transfer
- **Data Isolation**: Each user gets their own table they can modify without affecting others
- **Zero Storage Cost (Initially)**: Only stores differences when data is modified
- **Time Travel**: Clones preserve the source table's history
- **Test Environment**: Safe environment to experiment with data transformations

**How:** 
- **CLONE** keyword creates a metadata copy pointing to the same data
- Source: `DATA.STOCK_PRICE_HISTORY` (shared reference table)
- Target: `STOCK_PRICE_HISTORY` (in your personal schema)
- **OR REPLACE** ensures a clean slate if run multiple times

The cloned table contains historical OHLCV data:
- Date, Ticker symbol
- Open, High, Low, Close prices
- Volume, Dividends, Stock splits

**📖 Documentation:**
- [CREATE TABLE ... CLONE](https://docs.snowflake.com/en/sql-reference/sql/create-clone)
- [Zero-Copy Cloning](https://docs.snowflake.com/en/user-guide/tables-storage-considerations#label-cloning-tables)
- [Understanding Table Cloning](https://docs.snowflake.com/en/user-guide/object-clone)


In [ ]:
CREATE OR REPLACE TABLE STOCK_PRICE_HISTORY CLONE DATA.STOCK_PRICE_HISTORY;

### 📈 Examine Stock Price History Data

**What:** This cell queries and displays the stock price history data to verify the clone was successful.

**Why:** 
- Validates that the table contains the expected data
- Previews the data structure and content
- Helps understand what data is available for semantic model creation

**How:** Simple SELECT statement retrieves all records from the STOCK_PRICE_HISTORY table, showing historical OHLCV (Open, High, Low, Close, Volume) data with dividends and stock splits.

**📖 Documentation:**
- [Querying Data in Snowflake](https://docs.snowflake.com/en/user-guide/querying-data)


In [ ]:
SELECT * FROM STOCK_PRICE_HISTORY

## Create the Semantic View

This could also be done a number of different ways, mainly through Snowsight. If you have YAML already provisioned in this case you can simply use the system function as shown below.

NOTE: Make sure to change schema and database in the yaml below before creation to HOL.<YOUR_USER>

### 🧠 Create Semantic Model for Natural Language Queries

**What:** This cell creates a semantic view that enables natural language querying of stock price data using Cortex Analyst.

**Why:** 
- **Natural Language to SQL**: Users can ask questions in plain English instead of writing SQL
- **Business Context**: Adds meaning to data through descriptions, synonyms, and custom instructions
- **Cortex Analyst Integration**: Powers the Cortex Analyst tool that agents use to query structured data
- **Verified Queries**: Provides examples that improve query generation accuracy
- **Data Governance**: Centralizes business logic and metrics definitions

**How:** The YAML semantic model defines:

**1. Model Metadata:**
- Name and description of the semantic model
- Custom instructions for query interpretation (default filters, rounding, return calculations)

**2. Table Definition:**
- Maps logical "stock_prices" to physical `STOCK_PRICE_HISTORY` table
- ⚠️ **NOTE**: Update `schema: jacob_scott` to your username

**3. Time Dimensions:**
- `date` - Trading date with synonyms like "trading date", "day"

**4. Dimensions (Categorical Data):**
- `ticker` - Stock symbol (NVDA, AAPL, MSFT) marked as enum for better query planning

**5. Facts (Measurable Values):**
- Price data: open_price, high_price, low_price, close_price
- Trading data: volume, dividends, stock_splits
- Each with synonyms for natural language understanding

**6. Metrics (Aggregations):**
- `avg_close_price` - Average closing price over period
- `max_close_price` - Record high close
- `min_close_price` - Lowest close
- `total_volume` - Cumulative shares traded

**7. Filters (Reusable Predicates):**
- `last_30_days` - Past month filter
- `last_year` - Past 365 days filter

**8. Verified Queries (Training Examples):**
- Example questions with validated SQL
- Improves accuracy for similar questions

**Example Questions This Enables:**
- "What was the average closing price for NVDA last month?"
- "Show me the highest price for AAPL this year"
- "How much volume traded for MSFT in the last 30 days?"

**📖 Documentation:**
- [Cortex Analyst Semantic Model](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst/semantic-model-spec)
- [SYSTEM$CREATE_SEMANTIC_VIEW_FROM_YAML](https://docs.snowflake.com/en/sql-reference/functions/system_create_semantic_view_from_yaml)
- [Cortex Analyst Overview](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst)
- [Semantic Model Best Practices](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst/semantic-model-spec#label-semantic-model-best-practices)


In [ ]:
CALL SYSTEM$CREATE_SEMANTIC_VIEW_FROM_YAML(
  'HOL.' || $CURRENT_USER,
  $$
name: stock_prices_semantic_model
description: Semantic model for historical daily stock prices (OHLCV) with dividends and splits.
custom_instructions: >
  If the user doesn't specify a date filter, apply a default filter for the last year.
  Round numeric outputs to 2 decimal places.
  If the user asks for 'returns' or 'change', interpret as percentage change over
  the selected period (e.g., (last_close - first_close) / first_close * 100).

tables:
  - name: stock_prices
    description: Daily OHLCV per ticker.
    base_table:
      database: HOL
      schema: jacob_scott
      table: STOCK_PRICE_HISTORY

    time_dimensions:
      - name: date
        synonyms: ["trading date", "day"]
        description: Trading day (no time component).
        expr: DATE
        data_type: DATE
        unique: false

    dimensions:
      - name: ticker
        synonyms: ["symbol"]
        description: Stock ticker symbol (e.g., NVDA).
        expr: TICKER
        data_type: TEXT
        unique: false
        is_enum: true

    facts:
      - name: open_price
        synonyms: ["open"]
        description: Opening price for the day.
        expr: OPEN_PRICE
        data_type: NUMBER

      - name: high_price
        synonyms: ["high"]
        description: Intraday high price.
        expr: HIGH_PRICE
        data_type: NUMBER

      - name: low_price
        synonyms: ["low"]
        description: Intraday low price.
        expr: LOW_PRICE
        data_type: NUMBER

      - name: close_price
        synonyms: ["close", "closing price"]
        description: Closing price for the day.
        expr: CLOSE_PRICE
        data_type: NUMBER

      - name: volume
        synonyms: ["shares traded"]
        description: Number of shares traded during the day.
        expr: VOLUME
        data_type: NUMBER

      - name: dividends
        synonyms: ["dividend"]
        description: Cash dividends paid on the day.
        expr: DIVIDENDS
        data_type: NUMBER

      - name: stock_splits
        synonyms: ["split ratio"]
        description: Stock split ratio for the day (e.g., 4.0 == 4-for-1).
        expr: STOCK_SPLITS
        data_type: NUMBER

    metrics:
      - name: avg_close_price
        synonyms: ["average close"]
        description: Average closing price over the selected period.
        expr: AVG(CLOSE_PRICE)

      - name: max_close_price
        synonyms: ["record close", "all-time high close"]
        description: Maximum closing price over the selected period.
        expr: MAX(CLOSE_PRICE)

      - name: min_close_price
        synonyms: ["lowest close"]
        description: Minimum closing price over the selected period.
        expr: MIN(CLOSE_PRICE)

      - name: total_volume
        synonyms: ["sum volume", "trading volume"]
        description: Total shares traded over the selected period.
        expr: SUM(VOLUME)

    filters:
      - name: last_30_days
        synonyms: ["past month"]
        description: Filter to the last 30 calendar days.
        expr: "DATE >= DATEADD('day', -30, CURRENT_DATE())"

      - name: last_year
        synonyms: ["past year", "last 12 months"]
        description: Filter to the last 365 days.
        expr: "DATE >= DATEADD('day', -365, CURRENT_DATE())"

verified_queries:
  - name: average_close_by_month
    question: What was the average closing price by month for NVDA this year?
    sql: >
      SELECT DATE_TRUNC('month', DATE) AS month,
             AVG(CLOSE_PRICE) AS avg_close
      FROM SNOWFLAKE_INTELIGENCE_HOL.DATA.STOCK_PRICE_HISTORY
      WHERE TICKER = 'NVDA'
        AND DATE >= DATE_TRUNC('year', CURRENT_DATE())
      GROUP BY 1
      ORDER BY 1

  - name: total_volume_last_30_days
    question: How many shares traded for AAPL in the last 30 days?
    sql: >
      SELECT SUM(VOLUME) AS total_volume
      FROM SNOWFLAKE_INTELIGENCE_HOL.DATA.STOCK_PRICE_HISTORY
      WHERE TICKER = 'AAPL'
        AND DATE >= DATEADD('day', -30, CURRENT_DATE())
  $$
  -- Omit third parameter or set to FALSE to create the view
);

### 🔍 Test Cortex Search on Snowflake Documentation

**What:** This cell demonstrates querying a public Cortex Search Service containing Snowflake's official documentation.

**Why:** 
- **Live Example**: Shows how to query an existing Cortex Search Service
- **Documentation Search**: Access to searchable Snowflake documentation database
- **Pattern Template**: Demonstrates the query structure for your own search services
- **JSON Response**: Shows how search results are returned and structured

**How:** 
- Queries the `SNOWFLAKE_DOCUMENTATION.SHARED.CKE_SNOWFLAKE_DOCS_SERVICE` search service
- Natural language query: "how do i configure a default schema for a snowflake user account"
- Requests 3 types of information:
  - `chunk` - The text content containing the answer
  - `document_title` - Title of the documentation page
  - `source_url` - Link to the full documentation
- Limits to top 10 results
- Returns results as JSON array in the `results` field

**Use Case:** This public search service can help agents answer Snowflake technical questions by retrieving relevant documentation.

**📖 Documentation:**
- [Cortex Search SEARCH_PREVIEW](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/query-cortex-search-service)
- [Query Cortex Search Service](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/query-cortex-search-service#using-sql)


In [ ]:
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'SNOWFLAKE_DOCUMENTATION.SHARED.CKE_SNOWFLAKE_DOCS_SERVICE',
      '{
        "query": "how do i configure a default schema for a snowflake user account",
        "columns":[
            "chunk",
            "document_title",
            "source_url"
        ],
        "limit":10
      }'
  )
)['results'] as results
;



### 🐍 Parse SQL Search Results into Python

**What:** This cell converts the SQL query results from the previous cell into a pandas DataFrame for easier manipulation in Python.

**Why:** 
- **Python Integration**: Enables Python-based data analysis on search results
- **Snowflake Notebooks**: Demonstrates the `cells` API that references other cell outputs
- **DataFrame Format**: Makes results easier to process, filter, and visualize

**How:** 
- Uses `cells.TEST_SEARCH_SQL` to reference the previous SQL cell's output
- Calls `.to_pandas()` to convert Snowflake results to a pandas DataFrame
- Result contains the JSON search results in the `RESULTS` column

**📖 Documentation:**
- [Using Snowflake Notebooks](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-develop-run)
- [Cell References in Notebooks](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-develop-run#referencing-cells)


In [ ]:
 
result = cells.TEST_SEARCH_SQL.to_pandas()


### 👁️ Display Raw Search Results

**What:** This cell displays the raw DataFrame containing the search results.

**Why:** 
- **Data Inspection**: View the structure of the returned data
- **Debugging**: Verify the JSON is properly formatted
- **Understanding Output**: See how Snowflake packages search results

**How:** Simply outputs the `result` DataFrame variable, which Snowflake Notebooks automatically renders as a formatted table.

**📖 Documentation:**
- [Displaying Data in Notebooks](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-develop-run#displaying-output)


In [ ]:
result

### 🔄 Convert JSON Search Results to Structured DataFrame

**What:** This cell parses the JSON search results and converts them into a structured pandas DataFrame with columns for each field.

**Why:** 
- **Structured Access**: Converts nested JSON into tabular format with named columns
- **Analysis Ready**: Makes it easy to filter, sort, and analyze search results
- **Column-based Access**: Access individual fields like `df['chunk']` or `df['source_url']`

**How:** 
- `json.loads()` - Parses the JSON string from the first row's RESULTS column
- `pd.DataFrame()` - Converts the JSON array into a DataFrame
- Result has columns: `chunk`, `document_title`, `source_url`, and relevance scores

**Example Output Structure:**
```
| chunk | document_title | source_url | score |
|-------|---------------|------------|-------|
| text  | doc name      | https://... | 0.95 |
```

**📖 Documentation:**
- [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
- [Working with JSON in Python](https://docs.python.org/3/library/json.html)


In [ ]:
df = pd.DataFrame(json.loads(result["RESULTS"][0]))
df

[Query Cortex Search REST API](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/query-cortex-search-service#rest-api)

### 🌐 Cortex Search via REST API (Python Implementation)

**What:** This cell is a placeholder for implementing Cortex Search queries using the REST API from Python code.

**Why:** 
- **Alternative Access Method**: REST API allows querying Cortex Search from external applications
- **Integration**: Enables calling Cortex Search from web apps, microservices, or external Python scripts
- **Authentication**: Uses OAuth or JWT tokens for secure access
- **Programmatic Access**: Useful for building custom applications that need search functionality

**How:** To implement, you would:
1. Create a `CortexSearchService` object using the Core API
2. Call the `.search()` method with your query
3. Process the returned results

**Example Implementation:**
```python
from snowflake.core import Root
root = Root(session)
search_service = root.databases["HOL"].schemas[session.get_current_schema()].cortex_search_services["SEC_FILINGS_SEARCH"]

response = search_service.search(
    query="NVIDIA revenue guidance",
    columns=["DOCUMENT_NAME", "CHUNK_TEXT"],
    limit=5
)
```

**📖 Documentation:**
- [Query Cortex Search REST API](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/query-cortex-search-service#rest-api)
- [Snowflake Python API for Cortex Search](https://docs.snowflake.com/en/developer-guide/snowflake-python-api/snowflake-python-cortex-search)
- [REST API Authentication](https://docs.snowflake.com/en/developer-guide/sql-api/authenticating)


## Configure Cortex Agent

Now that we have all the pieces in place for the agent to pull context from, it's time to create the agent. The rest of the process will be done in Snowsight. Follow the steps below to complete the agent creation.


### Step-by-Step Guide
1. Create the Agent 
    - Go to AI & ML > Agents
    - Select "Create Agent"
    - Enter agent name and description
2. Add Cortex Search Service
    - Select "Tools" > "Cortex Search Services" > "+ Add"
    - Select your existing Cortex Search service
    - Enter chunk_id as ID column and document_name as title column.
    - Enter name and description for the search service
    - Click "Add"
3. Add Cortex Analyst Tool (Using Semantic Model)
    - Select "Tools" > "Cortex Analyst" > "+ Add"
    - Enter name for the semantic model tool
    - Select the semantic view
    - Choose warehouse for query execution
    - Set query timeout (seconds)
    - Add description
    - Click "Add"
4. Add Custom Tools
    - Select "Tools" > "Custom Tools" > "+ Add"
    - Leave resource type as procedure
    - Select existing stored procedure
    - Configure parameters (name, type, description, required status)
    - Select warehouse for execution (leave as default)
    - Add tool description and optionally argument descriptions
    - Click "Add"
5. Add Custom Prompt/Instructions
    - In the agent configuration, add custom instructions for:
        * Response behavior
        * Orchestration logic
        * System prompts
6. Save and Test
    - Click "Save" to create the agent
    - Test with sample questions
    - Monitor agent performance and user feedback